In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyscf
from qiskit_nature.second_q.drivers import PySCFDriver
import qiskit_nature
import qiskit_nature.units as units
import qiskit_nature.second_q.mappers as mapp

In [9]:
# Initialize Molecule
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 1.388949",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=units.DistanceUnit.BOHR)
hydrogen = driver.run()

# Get Hamiltonian
hamiltonian = hydrogen.hamiltonian

# Get Overlap Integrals
integrals = hamiltonian.electronic_integrals
integrals.alpha



PolynomialTensor({'+-': array([[-1.25633898e+00,  6.11240792e-17],
       [-4.24218158e-17, -4.71896108e-01]]), '++--': array([[[[ 6.75710125e-01,  2.55454894e-17],
         [ 1.32144979e-16,  1.80931209e-01]],

        [[ 6.51244823e-18,  1.80931209e-01],
         [ 6.64581703e-01, -1.24051783e-16]]],


       [[[-4.89138804e-18,  6.64581703e-01],
         [ 1.80931209e-01, -1.52325920e-16]],

        [[ 1.80931209e-01, -8.09973793e-17],
         [ 7.02372465e-17,  6.98573694e-01]]]])})

In [3]:
hydrogen.orbital_occupations

array([1., 0.])

In [10]:
# Convert to second quantization
sq_form = hamiltonian.second_q_op()
sq_form

FermionicOp({'+_0 +_0 -_0 -_0': 0.3378550623307603, '+_0 +_0 -_1 -_1': 0.09046560429576136, '+_0 +_1 -_0 -_1': 0.09046560429576121, '+_0 +_1 -_1 -_0': 0.33229085136554265, '+_0 +_2 -_2 -_0': 0.3378550623307603, '+_0 +_2 -_3 -_1': 0.09046560429576136, '+_0 +_3 -_2 -_1': 0.09046560429576121, '+_0 +_3 -_3 -_0': 0.33229085136554265, '+_1 +_0 -_0 -_1': 0.3322908513655427, '+_1 +_0 -_1 -_0': 0.0904656042957614, '+_1 +_1 -_0 -_0': 0.09046560429576125, '+_1 +_1 -_1 -_1': 0.3492868467800374, '+_1 +_2 -_2 -_1': 0.3322908513655427, '+_1 +_2 -_3 -_0': 0.0904656042957614, '+_1 +_3 -_2 -_0': 0.09046560429576125, '+_1 +_3 -_3 -_1': 0.3492868467800374, '+_2 +_0 -_0 -_2': 0.3378550623307603, '+_2 +_0 -_1 -_3': 0.09046560429576136, '+_2 +_1 -_0 -_3': 0.09046560429576121, '+_2 +_1 -_1 -_2': 0.33229085136554265, '+_2 +_2 -_2 -_2': 0.3378550623307603, '+_2 +_2 -_3 -_3': 0.09046560429576136, '+_2 +_3 -_2 -_3': 0.09046560429576121, '+_2 +_3 -_3 -_2': 0.33229085136554265, '+_3 +_0 -_0 -_3': 0.3322908513655427

In [5]:
# Get nuclear repulsion, since it is not included in the electronic hamiltonian above
hamiltonian.nuclear_repulsion_energy 

0.7199688397486157

In [6]:
BKconverter = mapp.QubitConverter(mapp.BravyiKitaevMapper())
JWconverter = mapp.QubitConverter(mapp.JordanWignerMapper())
Parityconverter = mapp.QubitConverter(mapp.ParityMapper())

In [7]:
BKHam = BKconverter.convert(sq_form)
JWHam = JWconverter.convert(sq_form)
ParityHam = Parityconverter.convert(sq_form)
print(BKHam)

-0.8105480324093168 * IIII
+ 0.17218390818952162 * IIIZ
- 0.2257534184802253 * IIZZ
+ 0.1209126235348907 * IIZI
+ 0.1721839081895214 * IZII
+ 0.16892753116538015 * IZIZ
+ 0.045232802147880655 * ZXIX
- 0.045232802147880655 * IXZX
- 0.045232802147880655 * ZXZX
+ 0.045232802147880655 * IXIX
- 0.2257534184802254 * ZZZI
+ 0.16614542568277135 * ZZZZ
+ 0.16614542568277132 * IZZZ
+ 0.1746434233900187 * ZZIZ
+ 0.1209126235348907 * ZIZI


In [8]:
print(JWHam)

-0.8105480324093168 * IIII
+ 0.17218390818952162 * IIIZ
- 0.2257534184802253 * IIZI
+ 0.1209126235348907 * IIZZ
+ 0.1721839081895214 * IZII
+ 0.16892753116538015 * IZIZ
+ 0.045232802147880655 * YYYY
+ 0.045232802147880655 * XXYY
+ 0.045232802147880655 * YYXX
+ 0.045232802147880655 * XXXX
- 0.2257534184802254 * ZIII
+ 0.16614542568277135 * ZIIZ
+ 0.16614542568277132 * IZZI
+ 0.1746434233900187 * ZIZI
+ 0.1209126235348907 * ZZII


In [11]:
print(ParityHam)

-0.8105480324093168 * IIII
+ 0.17218390818952162 * IIIZ
- 0.2257534184802253 * IIZZ
+ 0.1209126235348907 * IIZI
+ 0.1721839081895214 * IZZI
+ 0.16892753116538015 * IZZZ
+ 0.045232802147880655 * ZXIX
- 0.045232802147880655 * IXZX
- 0.045232802147880655 * ZXZX
+ 0.045232802147880655 * IXIX
- 0.2257534184802254 * ZZII
+ 0.16614542568277135 * ZZIZ
+ 0.16614542568277132 * IZIZ
+ 0.1746434233900187 * ZZZZ
+ 0.1209126235348907 * ZIZI
